In [1]:
!pip install transformers

In [2]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [4]:
!pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00


In [5]:
from finvizfinance.news import News

fnews = News()
all_news = fnews.get_news()

In [6]:
all_news['news'].head()

,Date,Title,Source,Link
0,08:13PM,Trump considers a 'blanket' option of 20% tariffs,finance.yahoo.com,https://finance.yahoo.com/news/the-economic-ri...
1,08:12PM,"Trump's tariffs will take effect immediately, ...",finance.yahoo.com,https://finance.yahoo.com/news/live/trump-tari...
2,08:11PM,Stock futures are little changed ahead of Trum...,www.cnbc.com,https://www.cnbc.com/2025/04/01/stock-market-t...
3,08:11PM,"Republicans eye $25,000 SALT cap as Trump's ta...",finance.yahoo.com,https://finance.yahoo.com/news/republicans-eye...
4,08:09PM,US stock futures rise as Trump's tariffs set s...,finance.yahoo.com,https://finance.yahoo.com/news/live/stock-mark...
